In [1]:
from model import *
from torch.optim import AdamW,Adam,SGD
from torch.nn.utils import clip_grad_value_

In [2]:
transform = A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=12, p=1)
data_train = ImgDataset('omniglot-py/images_background',n,k,64,transform)
data_train = DataLoader(data_train,b,drop_last=True)
data_val = ImgDataset('omniglot-py/images_evaluation',n,k,16,transform)
data_val = DataLoader(data_val,b,drop_last=True)

In [3]:
lr = 1e-3
clip = 1e-2
epochs = 10

In [4]:
model = AttenMeta_linearHead(d, dropout,layers,True).to('cuda')
#model.input_target.to('cuda')
paras = model.parameters()
#opt = AdamW(paras,lr = lr,amsgrad=True)
#opt = Adam(paras,lr = lr)
opt = SGD(paras,lr = lr)

In [5]:

opt.zero_grad()
for epoch in range(epochs):

    model.train()
    np.random.seed()
    train_loss = 0
    val_loss = 0
    
    # Training #
    for i,x in enumerate(data_train):
        x = x.to('cuda:0')
        loss = model(x,True)
        loss.backward()
        clip_grad_value_(paras,clip)
        opt.step()
        opt.zero_grad()
        train_loss += loss.item()
        
    # evaluating #
    model.eval()
    with torch.no_grad():
        for j,data in enumerate(data_val):
            x = x.to('cuda:0')
            loss = model(x,True)
            val_loss += loss.item()      
    print('epoch:{}, train_loss:{}, val_loss:{}'.format(epoch,train_loss/i,val_loss/j))

epoch:0, train_loss:1.0947913396156441, val_loss:0.5650018321143256
epoch:1, train_loss:0.4017358114153652, val_loss:0.18906085027588737
epoch:2, train_loss:0.21622159950814004, val_loss:0.12110351688332027
epoch:3, train_loss:0.14241701844385116, val_loss:0.06226181983947754
epoch:4, train_loss:0.11106859387482627, val_loss:0.052106790244579315
epoch:5, train_loss:0.08856173263768018, val_loss:0.04778070168362723
epoch:6, train_loss:0.07022271513686341, val_loss:0.03059788089659479
epoch:7, train_loss:0.06205005413394863, val_loss:0.026727262884378433
epoch:8, train_loss:0.05433122567453627, val_loss:0.021090687563021977
epoch:9, train_loss:0.05059084712953891, val_loss:0.021288653628693685


In [20]:
acc = 0
model.eval()
with torch.no_grad():
    for j,data in enumerate(data_val):
        x = x.to('cuda:0')
        yhat = model(x,False)
        acc += torch.sum(yhat.argmax(-1).flatten() == model.y)/b/n
print(acc/(j+1))

tensor(1., device='cuda:0')


In [19]:
acc/

tensor(10., device='cuda:0')

In [16]:
j

9

In [9]:
torch.sum(yhat.argmax(-1).flatten() == model.y)/b/n

tensor(1., device='cuda:0')

In [7]:
model.y.shape

torch.Size([192])

In [11]:
yhat

tensor([[ 5.7488, -1.0974, -2.4001, -0.8369, -1.4614, -2.3128],
        [-0.8348,  5.9136, -0.4287, -1.4588, -1.0326, -0.9763],
        [-1.6675, -0.3194,  6.3355, -1.1945, -0.9635, -0.4417],
        ...,
        [-1.1314, -1.0027, -1.6710,  5.1420, -0.9656, -0.6826],
        [-0.4006, -1.5854, -1.3414, -1.7718,  6.0901, -1.7882],
        [-0.7888, -0.7342, -1.7054, -1.4847, -1.2578,  4.7970]],
       device='cuda:0')

In [6]:
acc = 0
model.eval()
with torch.no_grad():
    for j,data in enumerate(data_val):
        x = x.to('cuda:0')
        yhat = model(x,False)
        acc += torch.sum(yhat.argmax(-1).flatten() == model.y)/b/n
print(acc/j)

tensor(0.3704, device='cuda:0')


In [7]:
1/n, -np.log(1/n)

(0.16666666666666666, 1.791759469228055)

In [7]:
yhat

tensor([[ 2.9969, -0.9699, -1.0318, -0.3555, -1.4680,  2.9855],
        [-5.1179,  0.7265,  0.6895,  0.6566,  0.6806, -2.0669],
        [-5.1179,  0.7265,  0.6895,  0.6566,  0.6806, -2.0669],
        ...,
        [-5.1178,  0.7264,  0.6896,  0.6564,  0.6806, -2.0670],
        [-5.1178,  0.7264,  0.6896,  0.6564,  0.6806, -2.0670],
        [ 2.9970, -0.9698, -1.0318, -0.3555, -1.4684,  2.9854]],
       device='cuda:0')